<a href="https://colab.research.google.com/github/kangwonlee/fdm_example/blob/main/fdm_si.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sample Python code simulating the behavior of a doped semiconductor using FDM:
### AskUp

This code simulates the behavior of a Si substrate with a doping profile of P impurities. It solves Poisson's equation and the continuity equation using FDM to calculate the electric potential and the electron and hole concentrations in the substrate. The simulation results are then plotted as a function of depth. Note that this is just a simple example, and more complex simulations may involve additional material properties and equations.
___
### ChatGPT

The code is trying to solve the continuity equations for electrons and holes, as well as Poisson's equation for the electric potential, in a semiconductor device with a doping profile. The semiconductor is assumed to be a p-n junction, with one side doped with acceptors (p-type) and the other side doped with donors (n-type).

The continuity equations describe how the concentration of electrons and holes change over time, and are given by:

$\frac{\partial n}{\partial t} = \nabla \cdot (D_n \nabla n) + G - R$

$\frac{\partial p}{\partial t} = \nabla \cdot (D_p \nabla p) + G - R$

where $n$ and $p$ are the electron and hole concentrations, $D_n$ and $D_p$ are their diffusion coefficients, $G$ is the generation rate of electron-hole pairs, and $R$ is the recombination rate.

Poisson's equation describes the relationship between the electric potential and the charge density, and is given by:

$\nabla^2 V = -\frac{\rho}{\epsilon}$

where $V$ is the electric potential, $\rho$ is the charge density, and $\epsilon$ is the permittivity of the material.

The simulation calculates the electric potential using Poisson's equation, and then uses the resulting potential to calculate the electron and hole concentrations using the continuity equations. The simulation then iterates over time to update the electric potential and the electron and hole concentrations until the simulation time is complete. Finally, the simulation results are plotted to show the concentration profiles of electrons and holes as a function of depth in the semiconductor.
___
### Partial Differential Equation and Partial Difference Equation

The code is implementing the Finite Difference Method (FDM) to solve a set of partial difference equations. The partial difference equations for electron and hole concentrations are discretized versions of the continuity equation:

$\frac{\partial n}{\partial t} = \frac{1}{q}\frac{\partial J_n}{\partial x} + G_{n} - R_{n}$

$\frac{\partial p}{\partial t} = \frac{1}{q}\frac{\partial J_p}{\partial x} + G_{p} - R_{p}$

where $n$ and $p$ are the electron and hole concentrations, $J_n$ and $J_p$ are the electron and hole currents, $G_n$ and $G_p$ are the generation rates of electrons and holes, and $R_n$ and $R_p$ are the recombination rates of electrons and holes.

Using forward differences for the time derivative and central differences for the spatial derivative, we can discretize the continuity equation as:

$\frac{n_i^{k+1} - n_i^k}{\Delta t} = \frac{1}{q}\frac{J_{n,i+1}^k - J_{n,i-1}^k}{2\Delta x} + G_{n,i} - R_{n,i}$

$\frac{p_i^{k+1} - p_i^k}{\Delta t} = \frac{1}{q}\frac{J_{p,i+1}^k - J_{p,i-1}^k}{2\Delta x} + G_{p,i} - R_{p,i}$

where $n_i^k$ and $p_i^k$ are the electron and hole concentrations at grid point $i$ and time step $k$, $J_{n,i}^k$ and $J_{p,i}^k$ are the electron and hole currents at grid point $i$ and time step $k$, and $G_{n,i}$, $G_{p,i}$, $R_{n,i}$, and $R_{p,i}$ are the generation and recombination rates at grid point $i$.

Next, we can discretize Poisson's equation using central differences for the second spatial derivative:

$\frac{\partial^2 V}{\partial x^2} = -\frac{\rho}{\epsilon}$

$\frac{V_{i+1} - 2V_i + V_{i-1}}{\Delta x^2} = -\frac{\rho_i}{\epsilon}$

where $V_i$ is the electric potential at grid point $i$, $\rho_i$ is the charge density at grid point $i$, and $\epsilon$ is the permittivity of the material.

Finally, we can solve for the unknowns $n_i^{k+1}$, $p_i^{k+1}$, and $V_i$ using the discretized equations above, along with appropriate boundary conditions.


In [ ]:
import numpy as np

# Define simulation parameters
dx = 1e-9  # grid spacing in meters
dt = 1e-12  # time step in seconds
t_total = 1e-9  # total simulation time in seconds
V_bias = 1  # applied bias in volts

# Define material properties
eps0 = 8.854e-12  # vacuum permittivity
eps_r = 11.68  # relative permittivity of Si
q = 1.602e-19  # elementary charge
k_B = 1.38e-23  # Boltzmann constant

# Define doping profile
N_d = 1e18  # doping concentration in cm^-3
x_d = 200e-9  # depth of doping profile in meters
N = N_d * np.exp(-np.arange(0, x_d, dx) / x_d)  # doping profile in cm^-3

# Initialize simulation variables
V = np.zeros(N.shape)  # electric potential in volts
n = np.zeros(N.shape)  # electron concentration in cm^-3
p = np.zeros(N.shape)  # hole concentration in cm^-3

# Define boundary conditions
V[0] = V_bias
n[0] = N_d
p[0] = 0
n[-1] = 0
p[-1] = N_d

# Define FDM coefficients
a = eps0 * eps_r / dx**2
b = q / eps0 / eps_r
c = dx**2 / q / eps0 / eps_r / k_B

# Perform simulation
for t in np.arange(0, t_total, dt):
    # Calculate electric potential using Poisson's equation
    V[1:-1] = (a * (V[0:-2] + V[2:]) + b * (N[1:-1] - n[1:-1] + p[1:-1])) / (2 * a + b * dx**2)

    # Calculate electron and hole concentrations using continuity equation
    n[1:-1] = n[1:-1] + dt * (N[1:-1] * np.exp((V[1:-1] - V_bias) / (k_B * c)) - n[1:-1] / c) / dx
    p[1:-1] = p[1:-1] + dt * (N[1:-1] * np.exp(-(V[1:-1] - V_bias) / (k_B * c)) - p[1:-1] / c) / dx

# Plot simulation results
import matplotlib.pyplot as plt
plt.plot(np.arange(0, x_d, dx) * 1e9, n, label='n')
plt.plot(np.arange(0, x_d, dx) * 1e9, p, label='p')
plt.xlabel('Depth (nm)')
plt.ylabel('Concentration (cm^-3)')
plt.legend()
plt.show()
